In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
in_filepath = '../Datasets/serie_A/whoscored_data/'

in_files = [
    'scores_2009-2010.csv',
    'scores_2010-2011.csv',
    'scores_2011-2012.csv',
    'scores_2012-2013.csv',
    'scores_2013-2014.csv',
    'scores_2014-2015.csv',
    'scores_2015-2016.csv',
    'scores_2016-2017.csv',
    'scores_2017-2018.csv'
]
in_filename = 'scores_2016-2017.csv'

loc = "../Datasets/serie_A/"
in_filestandings = loc + "ISAStandings.csv"

In [3]:
matches = pd.read_csv(in_filepath + in_filename)
matches.drop(['Unnamed: 0'],1, inplace=True)
Standings = pd.read_csv(in_filestandings)    
known_teams = Standings.Team

#print(known_teams)

In [4]:
def get_team_scores(team):
    coords = np.sort(team.coordinate.unique() ) # sorted from minor (goal keeper) to max (attack)
    #print(coords)
    team_score = np.round( sum(team.score), 3 )
    attack_score = 0
    medium_score = 0
    defense_score = 0
    gk_score = 0
    
    for i in range(len(coords)):
        s = sum( team.score[ team.coordinate == coords[i] ] )
        if i == 0:
            gk_score = s
        elif i == 1:
            defense_score = s
        elif i == 2:
            medium_score = s
        else:
            attack_score += s
                
    check = np.round( attack_score + medium_score + defense_score + gk_score, 3 ) 
    assert (team_score == check), "[ERROR] Team's score: calculated %f but should be %f"%(check, team_score)

    return attack_score, medium_score, defense_score, gk_score

In [5]:
print(known_teams.unique())
matches = pd.read_csv(in_filepath + in_filename)
matches.drop(['Unnamed: 0'],1, inplace=True)


for in_file in in_files:
    matches = pd.read_csv(in_filepath + in_file)
    #print(in_file, '\n',matches.HomeTeam.unique())
    teams = matches.HomeTeam.unique()
    for team in teams:
        if team not in list(known_teams.unique()):
            print('Unknown team: ', team)
    #print('=================')

['Ancona' 'Ascoli' 'Atalanta' 'Benevento' 'Bari' 'Bologna' 'Brescia'
 'Cagliari' 'Carpi' 'Catania' 'Cesena' 'Chievo' 'Como' 'Crotone' 'Empoli'
 'Fiorentina' 'Frosinone' 'Genoa' 'Inter' 'Juventus' 'Lazio' 'Lecce'
 'Livorno' 'Messina' 'Milan' 'Modena' 'Napoli' 'Novara' 'Palermo' 'Parma'
 'Perugia' 'Pescara' 'Piacenza' 'Reggina' 'Roma' 'Torino' 'Treviso'
 'Sampdoria' 'Sassuolo' 'Siena' 'Spal' 'Udinese' 'Venezia' 'Verona'
 'Vicenza']
Unknown team:  Robur Siena
Unknown team:  AC Milan
Unknown team:  AC Milan
Unknown team:  AC Milan
Unknown team:  Robur Siena
Unknown team:  AC Milan
Unknown team:  Robur Siena
Unknown team:  AC Milan
Unknown team:  AC Milan
Unknown team:  AC Milan
Unknown team:  AC Milan
Unknown team:  SPAL
Unknown team:  AC Milan


In [37]:
def aggregate_scores(matches):
    teams_dict = {
        'AC Milan': 'Milan',
        'Robur Siena': 'Siena',
        'SPAL': 'Spal'
    }
    
    teams_scores = pd.DataFrame(columns = ['Date', 'HomeTeam', 'AwayTeam', 'HomeAttack', 'HomeMedium', 'HomeDefense', 'HomeGK', 'AwayAttack', 'AwayMedium', 'AwayDefense', 'AwayGK'])

    ht = None
    at = None
    for index, row in matches.iterrows():
        if row.HomeTeam != ht and row.AwayTeam != at:
            scores = matches[(matches.HomeTeam == row.HomeTeam) & (matches.AwayTeam == row.AwayTeam)]
        
            home_team = scores[ scores.team.str.contains('home') ]
            HomeAttack, HomeMedium, HomeDefense, HomeGK = get_team_scores(home_team)
        
            away_team = scores[ scores.team.str.contains('away') ]
            AwayAttack, AwayMedium, AwayDefense, AwayGK = get_team_scores(away_team)
        
            ht_name = home_team.HomeTeam.iloc[0]
            at_name = away_team.AwayTeam.iloc[0]
            # uniform names with those on other databases
            if ht_name in teams_dict.keys():
                ht_name = teams_dict[ht_name]
            if at_name in teams_dict.keys():
                at_name = teams_dict[at_name]
        
            s = pd.DataFrame.from_dict([{
                'Date' : home_team.date.iloc[0],
                'HomeTeam' : ht_name,
                'AwayTeam' : at_name,
                'HomeAttack' : HomeAttack,
                'HomeMedium' : HomeMedium,
                'HomeDefense' : HomeDefense,
                'HomeGK' : HomeGK,
                'AwayAttack' : AwayAttack,
                'AwayMedium' : AwayMedium,
                'AwayDefense' : AwayDefense,
                'AwayGK' : AwayGK}])
            # [home_team.date.iloc[0], ht_name, at_name, HomeAttack, HomeMedium, HomeDefense, HomeGK, AwayAttack, AwayMedium, AwayDefense, AwayGK]
        
            teams_scores = teams_scores.append(s, ignore_index=True)
            ht = row.HomeTeam
            at = row.AwayTeam
            
    return teams_scores


teams_scores = aggregate_scores(matches)
display(teams_scores)
        

,AwayAttack,AwayDefense,AwayGK,AwayMedium,AwayTeam,Date,HomeAttack,HomeDefense,HomeGK,HomeMedium,HomeTeam
0,18.8,25.0,6.1,18.5,Cagliari,2017-08-19,31.7,30.6,8.2,14.3,Juventus
1,24.7,26.4,6.6,21.6,Napoli,2017-08-19,19.4,24.0,8.4,18.4,Verona
2,19.6,30.6,8.0,21.3,Roma,2017-08-20,20.6,20.9,5.6,26.6,Atalanta
3,27.8,26.2,6.6,14.1,Torino,2017-08-20,21.9,26.6,5.4,20.0,Bologna
4,23.8,29.4,7.4,22.6,Milan,2017-08-20,12.3,23.8,7.0,25.1,Crotone
5,21.2,24.7,6.3,18.2,Fiorentina,2017-08-20,31.4,28.9,7.6,13.6,Inter
6,12.6,21.6,8.5,36.8,Spal,2017-08-20,12.8,21.9,7.0,36.0,Lazio
7,13.5,26.6,7.2,28.0,Benevento,2017-08-20,22.9,27.1,7.1,21.4,Sampdoria
8,19.5,20.5,7.0,27.5,Genoa,2017-08-20,19.7,29.1,7.7,20.2,Sassuolo
9,23.1,26.6,6.8,21.3,Chievo,2017-08-20,13.9,27.0,5.6,27.0,Udinese


In [ ]:
scores = matches[(matches.HomeTeam == 'Bologna') & (matches.AwayTeam == 'Torino')]
#print(scores)

indx = scores.team.str.contains('home')
team = scores[indx]

print(team.HomeTeam.iloc[0])
print(get_teams_scores(team))
            



In [36]:
print(team)

Cagliari
